In [2]:
import jellyfish
from collections import OrderedDict
import igraph as ig
import re
from pprint import pprint

def parse_entry(txt):
    head_line = re.compile(r'(?P<head>[A-Z0-9]{2})\s(?P<line>.+)')
    lines = txt.split('\n')
    data = OrderedDict()
    section = "UNKW"

    for line, next in zip(lines, lines[1:] + ['']):
        match_line = head_line.match(line)
        match_next = head_line.match(next)
        if match_line is not None:
            section = match_line.group('head')
            if match_next is not None or next == '':
                data[section] = match_line.group('line')
            else:
                data[section] = [match_line.group('line')]
        else:
            if section in data:
                data[section].append(line.strip())
            else:
                data[section] = [line.strip()]

    return data

def get_entry_label(entry):
    from string import punctuation

    label_parts = []

    mask = dict(zip(list(punctuation), [None] * len(punctuation)))
    clear = lambda s: s.translate(mask)

    if isinstance(entry['AU'], str):
        author = entry['AU']
    else:
        author = entry['AU'][0]

    first_name = author.split(', ')[0]
    last_name = author.split(', ')[-1]

    label_parts.append(first_name + ' ' + clear(last_name))
    label_parts.append(entry.get('PY', ''))
    label_parts.append(entry.get('J9', ''))
    label_parts.append('V' + entry.get('VL', ''))
    if 'BP' in entry:
        label_parts.append('P' + entry['BP'])
    elif 'AR' in entry:
        label_parts.append('p' + entry['AR'].upper())
    if 'DI' in entry:
        label_parts.append('DOI ' + entry['DI'])

    return ', '.join(label_parts)

def get_referenced_labels(entry):
    if 'CR' in entry:
        references = entry['CR']
        if isinstance(references, str):
            return [references, ]
        else:
            return references
    else:
        return []

def get_label_list(entries):
    labels = []

    for entry in entries:
        labels.append(get_entry_label(entry))
        labels.extend(get_referenced_labels(entry))

    return sorted(list(set(labels)))

def extract_edge_relations(entries):
    edges = []

    for entry in entries:
        label = get_entry_label(entry)
        references = get_referenced_labels(entry)
        edges.extend(zip([label] * len(references), references))

    return edges

def detect_duplicate_labels(labels,
                            similarity=jellyfish.jaro_winkler,
                            shared_first_letters=2,
                            threshold=0.96,
                            inverted=False):
    sorted_labels = sorted(labels)
    num_labels = len(sorted_labels)
    duplicates = dict()

    if inverted:
        comp = lambda x, y: x < y
    else:
        comp = lambda x, y: x > y

    for i in range(num_labels):
        label = duplicates.get(sorted_labels[i], sorted_labels[i])
        start_letters = label[:shared_first_letters]

        for j in range(i + 1, num_labels):
            other = sorted_labels[j]

            if not other.startswith(start_letters):
                break

            sim = similarity(label, other)

            if comp(sim, threshold):
                duplicates[other] = label

    return duplicates

def patch_list(items, patch):
    return list(map(patch.get, items, items))

def patch_tuple_list(items, patch):
    keys, values = zip(*items)
    keys = patch_list(keys, patch)
    values = patch_list(values, patch)
    return list(zip(keys, values))

class TreeOfScience():

    def __init__(self, path):
        self.graph = ig.Graph()
        self.path = path
        self.configure()

    def __build_graph(self):
        entries = list(
            map(
                parse_entry,
                open(self.path, 'r').read().split('\nER\n\n')
            )
        )[:-1]
        labels = get_label_list(entries)

        duplicates = detect_duplicate_labels(labels)
        unique_labels = list(set(patch_list(labels, duplicates)))
        edge_relations = extract_edge_relations(entries)
        unique_edge_relations = list(set(
            patch_tuple_list(edge_relations, duplicates)
        ))

        identifiers = dict(zip(unique_labels, range(len(unique_labels))))
        self.graph = ig.Graph(
            patch_tuple_list(unique_edge_relations, identifiers),
            directed=True
        )

        self.graph.vs['label'] = unique_labels

        # ig.plot(self.graph, 'img.png', bbox=(0, 0, 5000, 5000))

    def __preprocess_graph(self):
        # Filter out vertices that are not relevant in the dataset
        valid_vs = self.graph.vs.select(
            lambda v: v.indegree() != 1 or v.outdegree() != 0).indices
        self.graph = self.graph.subgraph(valid_vs)

        # Isolate the giant component
        self.graph = self.graph.clusters(ig.WEAK).giant()

    def __post_processg_graph(self):
        self.graph.vs['betweenness'] = self.graph.betweenness()
        self.graph.es['betweenness'] = self.graph.edge_betweenness()

    def configure(self):
        """Configures the :class:`~tos.graph.tree_of_science.TreeOfScience`
        instance according to
        its configuration and a given `data` field given in the config
        dictionary, it creates a graph from the edge relations of the
        entries contained in the data, filters the graph purging unnecesary
        vertices, and then extract the giant component of the graph.
        In a postprocessing stage it adds the edge and vertex betweenness
        to the graphs properties.
        """
        self.__build_graph()
        self.__preprocess_graph()
        self.__post_processg_graph()
        self.graph.write('file.graphml')

    def root(self, offset=0, count=10):
        """Computes the nodes in the root of the graph using
        the following criteria: nodes with high in degree, and zero
        out degree are in the root set, this function returns
        `count` nodes after the `offset` element according to this
        criteria.

        :param int offset: rank of the first node to return
        :param int count: total number of nodes to return
        :returns: sequense of nodes in the root
        :rtype: :class:`~igraph.VertexSeq`
        """
        from operator import itemgetter

        valid_vs = self.graph.vs.select(_outdegree_eq=0).indices
        items = zip(
            self.graph.vs[valid_vs].indices,
            self.graph.vs[valid_vs].indegree(),
            self.graph.vs[valid_vs].outdegree(),
        )

        sorted_items = sorted(items, key=itemgetter(1), reverse=True)
        indices = list(zip(*sorted_items))[0][offset:offset + count]

        return indices

    def trunk(self, offset=0, count=10):
        """Returns `count` nodes after the `ofset` belonging to the trunk
        of the graph, the *trunk degree* is computed according the
        following criteria:

        Compute the vertex betweness a of the graph and return them in
        descending order.

        @TODO: Relate the offset and count with the central point
        dominance.

        :param int offset: rank of the first node to return
        :param int count: total number of nodes to return
        :returns: sequense of nodes in the trunk
        :rtype: :class:`~igraph.VertexSeq`
        """
        from operator import itemgetter

        items = zip(
            self.graph.vs.indices,
            self.graph.vs['betweenness'],
        )

        sorted_items = sorted(items, key=itemgetter(1), reverse=True)
        indices = list(zip(*sorted_items))[0][offset:offset + count]
        return indices

    def branch(self, offset=0, count=10):
        raise NotImplementedError('This feature is not implemented yet')

    def leave(self, offset=0, count=10):
        """Computes the leave nodes in the graph using
        the following criteria: nodes with high out egree, and zero
        in degree are in the leave set, this function returns
        `count` nodes after the `offset` element according to this
        criteria.

        :param int offset: rank of the first node to return
        :param int count: total number of nodes to return
        :returns: sequense of leave nodes
        :rtype: :class:`~igraph.VertexSeq`
        """
        from operator import itemgetter

        valid_vs = self.graph.vs.select(_indegree_eq=0).indices
        items = zip(
            self.graph.vs[valid_vs].indices,
            self.graph.vs[valid_vs].indegree(),
            self.graph.vs[valid_vs].outdegree(),
        )

        sorted_items = sorted(items, key=itemgetter(2), reverse=True)
        indices = list(zip(*sorted_items))[0][offset:offset + count]

        return indices

if __name__ == '__main__':
    #__________________________________________________________________________
    tree = TreeOfScience('EM.txt')  # <AQUI EL ARCHIVO!!!!!!!!!!!!
    #^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    print('Leave:')
    pprint([tree.graph.vs['label'][x] for x in tree.leave()])
    print('Trunk:')
    pprint([tree.graph.vs['label'][x] for x in tree.trunk()])
    print('Root:')
    pprint([tree.graph.vs['label'][x] for x in tree.root()])

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.